In [1]:
import os
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as opt
import torchvision
import torchvision.datasets as data
import torchvision.transforms as trans
from torch.utils.tensorboard import SummaryWriter as summary
from torch.utils.data import DataLoader
from torch.autograd import backward
import matplotlib.pyplot as plt

In [2]:
# CREATE DIRECTORIES USED IN THIS PROGRAM HERE

if not os.path.exists("data/MNIST"):
    os.makedirs("data/MNIST")

if not os.path.exists("data/GAN/saved"):
    os.makedirs("data/GAN/saved")

if not os.path.exists("data/GAN/logs"):
    os.makedirs("data/GAN/logs")

In [20]:
# Set device to enable CUDA if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Set learning rate
learning_rate = 1e-4

# Latent space dimension
z = 64

# Image size (MNIST images are 28*28 pixels = 784)
image_size = 784

# Number of images in batch
batch_size = 64

# Set number of epochs
epochs = 250

# Directory configuration
dataset_dir  = "data/MNIST"
output_dir   = "data/GAN/"

In [4]:
# Discriminator -- distinguish images from ones it has seen already
# Currently: 2 layers, ReLU (mx = 0.1) and sigmoidal (output [0,1]) activation functions 

class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.d = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128,1),
            nn.Sigmoid(),
        )
    def forward(self,x):
        return self.d(x)
    
# Generator -- produce new images to try and fool the discriminator
# Currently: 2 layers, ReLU (mx = 0.1) and horizontal tangent ()
class Generator(nn.Module):
    def __init__(self, z, image_size):
        super().__init__()
        self.g = nn.Sequential(
            nn.Linear(z, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, image_size),
            nn.Tanh(),
        )
    def forward(self,x):
        return self.g(x)

In [5]:
d = Discriminator(image_size).to(device)
g = Generator(z, image_size).to(device)


set_noise = torch.randn((batch_size, z)).to(device)

transform = trans.Compose([trans.ToTensor(),trans.Normalize((0.5),(0.5))])

dataset   = data.MNIST(root='data/MNIST/', transform=transform, download=True)
loader    = DataLoader(dataset, batch_size=batch_size, shuffle=True)

d_optim   = opt.Adam(d.parameters(), lr=learning_rate)
g_optim   = opt.Adam(g.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

# Save real and generated images 
fake_imgs = summary(f"data/GAN/logs/fake")
real_imgs = summary(f"data/GAN/logs/real")

step_size = 0

In [21]:
# Reset step count (tracked at end of loop)
step = 0

# Run training
for epoch in range(epochs):
    for i, (real, labels) in enumerate(loader):
        real = real.view(-1, image_size).to(device)
        batch_size = real.shape[0]
# Train Discriminator
        gen_noise   = torch.randn((batch_size, z)).to(device)
        generated   = g(gen_noise)
    # Real images      
        d_real      = d(real).view(-1)
        d_real_loss = criterion(d_real, torch.ones_like(d_real))
    # Fake images
        d_gen_      = d(generated).view(-1)
        d_gen__loss = criterion(d_gen_, torch.zeros_like(d_gen_))
    # Calculate loss
        D_loss      = (d_real_loss+d_gen__loss)/2
    # Reset gradient, backpropagate, optimize
        d.zero_grad()
        D_loss.backward(retain_graph=True)
        d_optim.step()
# Train Generator
        output = d(generated).view(-1)
        G_loss = criterion(output, torch.ones_like(output))
    # Reset gradient, backpropagate, optimize
        g.zero_grad()
        G_loss.backward()
        g_optim.step()
        if i == 0:
            print(f"Epoch Number: [{epoch}/{epochs}]; Batch {i}/{len(loader)}; Discriminator Loss: {D_loss:.4f}, Generator Loss: {G_loss:.4f}")
            with torch.no_grad():
                generated = g(set_noise).reshape(-1,1,28,28)
                data = real.reshape(-1,1,28,28)
                gen__img_grid = torchvision.utils.make_grid(generated, normalize = True)
                real_img_grid = torchvision.utils.make_grid(data, normalize = True)
                fake_imgs.add_image("MNIST generated images", gen__img_grid, global_step=step)
                real_imgs.add_image("MNIST real images", real_img_grid, global_step=step)
                step += 1

Epoch Number: [0/250]; Batch 0/938; Discriminator Loss: 0.3876, Generator Loss: 1.8323
Epoch Number: [1/250]; Batch 0/938; Discriminator Loss: 0.3044, Generator Loss: 2.0361
Epoch Number: [2/250]; Batch 0/938; Discriminator Loss: 0.3396, Generator Loss: 1.9707
Epoch Number: [3/250]; Batch 0/938; Discriminator Loss: 0.2968, Generator Loss: 1.9761
Epoch Number: [4/250]; Batch 0/938; Discriminator Loss: 0.2596, Generator Loss: 2.1557
Epoch Number: [5/250]; Batch 0/938; Discriminator Loss: 0.3453, Generator Loss: 2.1991
Epoch Number: [6/250]; Batch 0/938; Discriminator Loss: 0.3818, Generator Loss: 1.7871
Epoch Number: [7/250]; Batch 0/938; Discriminator Loss: 0.4309, Generator Loss: 1.8748
Epoch Number: [8/250]; Batch 0/938; Discriminator Loss: 0.5310, Generator Loss: 1.9068
Epoch Number: [9/250]; Batch 0/938; Discriminator Loss: 0.5965, Generator Loss: 1.6049
Epoch Number: [10/250]; Batch 0/938; Discriminator Loss: 0.6560, Generator Loss: 1.5232
Epoch Number: [11/250]; Batch 0/938; Discr

Epoch Number: [94/250]; Batch 0/938; Discriminator Loss: 0.4302, Generator Loss: 1.3008
Epoch Number: [95/250]; Batch 0/938; Discriminator Loss: 0.5387, Generator Loss: 1.1230
Epoch Number: [96/250]; Batch 0/938; Discriminator Loss: 0.5644, Generator Loss: 1.4317
Epoch Number: [97/250]; Batch 0/938; Discriminator Loss: 0.4727, Generator Loss: 1.5491
Epoch Number: [98/250]; Batch 0/938; Discriminator Loss: 0.5840, Generator Loss: 1.2995
Epoch Number: [99/250]; Batch 0/938; Discriminator Loss: 0.6108, Generator Loss: 1.2319
Epoch Number: [100/250]; Batch 0/938; Discriminator Loss: 0.5626, Generator Loss: 1.2219
Epoch Number: [101/250]; Batch 0/938; Discriminator Loss: 0.6012, Generator Loss: 1.4123
Epoch Number: [102/250]; Batch 0/938; Discriminator Loss: 0.5782, Generator Loss: 1.2345
Epoch Number: [103/250]; Batch 0/938; Discriminator Loss: 0.5654, Generator Loss: 1.1988
Epoch Number: [104/250]; Batch 0/938; Discriminator Loss: 0.4869, Generator Loss: 1.2649
Epoch Number: [105/250]; Ba

Epoch Number: [187/250]; Batch 0/938; Discriminator Loss: 0.5007, Generator Loss: 1.1164
Epoch Number: [188/250]; Batch 0/938; Discriminator Loss: 0.5355, Generator Loss: 1.3347
Epoch Number: [189/250]; Batch 0/938; Discriminator Loss: 0.4781, Generator Loss: 1.3477
Epoch Number: [190/250]; Batch 0/938; Discriminator Loss: 0.4309, Generator Loss: 1.3709
Epoch Number: [191/250]; Batch 0/938; Discriminator Loss: 0.5156, Generator Loss: 1.1805
Epoch Number: [192/250]; Batch 0/938; Discriminator Loss: 0.5224, Generator Loss: 1.3294
Epoch Number: [193/250]; Batch 0/938; Discriminator Loss: 0.4985, Generator Loss: 1.3438
Epoch Number: [194/250]; Batch 0/938; Discriminator Loss: 0.5541, Generator Loss: 1.0952
Epoch Number: [195/250]; Batch 0/938; Discriminator Loss: 0.5166, Generator Loss: 1.4949
Epoch Number: [196/250]; Batch 0/938; Discriminator Loss: 0.5257, Generator Loss: 1.2748
Epoch Number: [197/250]; Batch 0/938; Discriminator Loss: 0.5532, Generator Loss: 1.3665
Epoch Number: [198/25

In [6]:
%load_ext tensorboard


In [18]:
%tensorboard --logdir="C:/Users/Nick/data/GAN/logs"

Reusing TensorBoard on port 6006 (pid 11352), started 0:29:50 ago. (Use '!kill 11352' to kill it.)

In [22]:
%tensorboard !kill 12188

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
2020-12-11 14:58:19.596244: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-12-11 14:58:19.601198: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
usage: tensorboard [-h] [--helpfull] {serve,dev} ...
tensorboard: error: invalid choice: '!kill' (choose from 'serve', 'dev')